# Segmenting and Clustering Neighborhoods in Toronto

### Data frame preparation

In [1]:
#import libraries

import requests
import lxml.html as lh
import pandas as pd
import numpy as np

In [2]:
# Get table from WIKI page

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)

# page contains 3 tables, we need first one
df = dfs[0]

In [3]:
# clean the data
# Condition 1: Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)
df.reset_index(inplace=True, drop=True)
# Condition 2: If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
df['Neighbourhood'] = np.where((df.Neighbourhood == 'Not assigned'),df.Borough,df.Neighbourhood)


In [4]:
# Check the structure
df.head()


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
# Check the shape
df.shape

(103, 3)

### Geocoder data collection 

[Google geocoder doesn't work without API key and for free since 2018](https://cloud.google.com/maps-platform/user-guide/?_ga=2.124905161.212364110.1538961567-1131107758.1527643759), so using OpenCage 



In [6]:
#install needed package
!pip install opencage

In [7]:
from opencage.geocoder import OpenCageGeocode
import time

Next cell will be hidden as contains secret keys:

key: OpenCage key

CLIENT_ID: Foursquare ID

CLIENT_SECRET: Foursquare Secret

VERSION: Foursquare API version

LIMIT: default Foursquare API limit value

In [8]:
# The code was removed by Watson Studio for sharing.

In [9]:
# Add coordinates
df['Latitude'] = 0.0
df['Longitude'] = 0.0

geocoder = OpenCageGeocode(key)

for index, row in df.iterrows():
    results = geocoder.geocode('{}, Toronto, Ontario'.format(row["Postal Code"]))
    df.loc[index,'Latitude'] = results[0]['geometry']['lat']
    df.loc[index,'Longitude'] = results[0]['geometry']['lng']
    time.sleep(1) #no more than 1 request per second




In [10]:
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.653482,-79.383935
1,M4A,North York,Victoria Village,43.727600,-79.314800
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655500,-79.362600
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722300,-79.450400
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.653482,-79.383935


### Toronto area exploration 

In [11]:
!pip install folium
import folium

address = 'Toronto, Canada'

results = geocoder.geocode('Toronto, Ontario')
tor_latitude = results[0]['geometry']['lat']
tor_longitude = results[0]['geometry']['lng']
print('The geograpical coordinates of Toronto are {}, {}.'.format(tor_latitude, tor_longitude))

     |████████████████████████████████| 94 kB 5.5 MB/s  eta 0:00:01
The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


### Create a map of areas

In [12]:

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[tor_latitude, tor_longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '({}), {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Getting venue data for each area

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

#### There are venues with category = 'Neighborhood', we don't need them, so remowing them

In [15]:
# remove venues with category Neighborhood

toronto_venues.drop(toronto_venues[toronto_venues['Venue Category'] == 'Neighborhood'].index, inplace=True)
toronto_venues.shape

(2517, 7)

In [16]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,Parkwoods,43.653482,-79.383935,Nathan Phillips Square,43.652270,-79.383516,Plaza
2,Parkwoods,43.653482,-79.383935,Japango,43.655268,-79.385165,Sushi Restaurant
3,Parkwoods,43.653482,-79.383935,Poke Guys,43.654895,-79.385052,Poke Place
4,Parkwoods,43.653482,-79.383935,Textile Museum of Canada,43.654396,-79.386500,Art Museum
5,Parkwoods,43.653482,-79.383935,Chatime 日出茶太,43.655542,-79.384684,Bubble Tea Shop


#### One hot encoding for all venues. Actually not for all, for limited number as per forsquare limitation

In [17]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,...,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Parkwoods,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Grouping data

In [18]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,...,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.041667,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.041667,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
92,Woburn,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
93,Woodbine Heights,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
94,York Mills West,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(100)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Breakfast Spot,Latin American Restaurant,Badminton Court,Newsagent,Eastern European Restaurant
1,"Alderwood, Long Branch",Convenience Store,Pizza Place,Pub,Coffee Shop,Pharmacy
2,"Bathurst Manor, Wilson Heights, Downsview North",Middle Eastern Restaurant,Pizza Place,Mediterranean Restaurant,Other Repair Shop,Deli / Bodega
3,Bayview Village,Flower Shop,Trail,Gas Station,Park,Event Space
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Comfort Food Restaurant,Toy / Game Store
...,...,...,...,...,...,...
91,"Willowdale, Willowdale West",Park,Bookstore,Baby Store,Coffee Shop,Yoga Studio
92,Woburn,Indian Restaurant,Coffee Shop,Taco Place,Park,Yoga Studio
93,Woodbine Heights,Convenience Store,Spa,Beer Store,Video Store,Fast Food Restaurant
94,York Mills West,Convenience Store,Park,Donut Shop,Flower Shop,Flea Market


### Run clustering k-means algorithm using top5 categories for the area

In [32]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0], dtype=int32)

#### Merge data and data preparation for map plot

In [33]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,0,Agincourt,Breakfast Spot,Latin American Restaurant,Badminton Court,Newsagent,Eastern European Restaurant
1,0,"Alderwood, Long Branch",Convenience Store,Pizza Place,Pub,Coffee Shop,Pharmacy
2,0,"Bathurst Manor, Wilson Heights, Downsview North",Middle Eastern Restaurant,Pizza Place,Mediterranean Restaurant,Other Repair Shop,Deli / Bodega
3,1,Bayview Village,Flower Shop,Trail,Gas Station,Park,Event Space
4,0,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Comfort Food Restaurant,Toy / Game Store


In [55]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Neighborhood'] == "Upper Rouge"]

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue


#### For some areas no venue were provided at all. Such data will be removed

In [50]:
toronto_merged = df
toronto_merged.head(100) 


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.653482,-79.383935
1,M4A,North York,Victoria Village,43.727600,-79.314800
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655500,-79.362600
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722300,-79.450400
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.653482,-79.383935
...,...,...,...,...,...
95,M1X,Scarborough,Upper Rouge,43.834000,-79.206900
96,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.668027,-79.369282
97,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.649200,-79.382300
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.651800,-79.507600


#### Merge aggregated and clustering data with original geo data to plot a map

In [51]:
# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')



In [38]:
toronto_merged.head(100) # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M3A,North York,Parkwoods,43.653482,-79.383935,0.0,Clothing Store,Coffee Shop,Theater,Plaza,Diner
1,M4A,North York,Victoria Village,43.727600,-79.314800,0.0,Hockey Arena,Portuguese Restaurant,Pizza Place,Intersection,Coffee Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655500,-79.362600,0.0,Coffee Shop,Breakfast Spot,Yoga Studio,Italian Restaurant,Food Truck
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722300,-79.450400,0.0,Clothing Store,Coffee Shop,Women's Store,Restaurant,Sandwich Place
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.653482,-79.383935,0.0,Clothing Store,Coffee Shop,Theater,Plaza,Diner
...,...,...,...,...,...,...,...,...,...,...,...
95,M1X,Scarborough,Upper Rouge,43.834000,-79.206900,NaN,NaN,NaN,NaN,NaN,NaN
96,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.668027,-79.369282,0.0,Coffee Shop,Pizza Place,Bakery,Café,Gym / Fitness Center
97,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.649200,-79.382300,0.0,Coffee Shop,Hotel,Café,Restaurant,Gym
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.651800,-79.507600,0.0,Breakfast Spot,Sushi Restaurant,Bank,Bar,Boutique


In [52]:
import matplotlib.cm as cm
import matplotlib.colors as colors

toronto_merged.dropna(subset = ["Cluster Labels"], inplace=True) #Removed empty rows

# create map
map_clusters = folium.Map(location=[tor_latitude, tor_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Conclusions

- clearly seen that most of the areas are falling to cluster 0 or cluster 1 which means that in terms of popular venues Toronto is devided by two main areas

Cluster 0 is more about shopping on a first glance



In [61]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M3A,North York,Parkwoods,43.653482,-79.383935,0.0,Clothing Store,Coffee Shop,Theater,Plaza,Diner
1,M4A,North York,Victoria Village,43.727600,-79.314800,0.0,Hockey Arena,Portuguese Restaurant,Pizza Place,Intersection,Coffee Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655500,-79.362600,0.0,Coffee Shop,Breakfast Spot,Yoga Studio,Italian Restaurant,Food Truck
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722300,-79.450400,0.0,Clothing Store,Coffee Shop,Women's Store,Restaurant,Sandwich Place
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.653482,-79.383935,0.0,Clothing Store,Coffee Shop,Theater,Plaza,Diner
...,...,...,...,...,...,...,...,...,...,...,...
97,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.649200,-79.382300,0.0,Coffee Shop,Hotel,Café,Restaurant,Gym
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.651800,-79.507600,0.0,Breakfast Spot,Sushi Restaurant,Bank,Bar,Boutique
99,M4Y,Downtown Toronto,Church and Wellesley,43.665600,-79.383000,0.0,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Fast Food Restaurant,Restaurant
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.780400,-79.250500,0.0,Italian Restaurant,Coffee Shop,Restaurant,Yoga Studio,Breakfast Spot


#### Cluster 1 is about Sport, parks and playgrounds (on a first glance, needed to be explored more in detail)

In [62]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.666200,-79.528200,1.0,Pharmacy,Grocery Store,Home Service,Skating Rink,Bank
7,M3B,North York,Don Mills,43.745000,-79.359000,1.0,Gym,Park,River,Trail,Furniture / Home Store
13,M3C,North York,Don Mills,43.733400,-79.332900,1.0,Gym,Park,River,Trail,Furniture / Home Store
14,M4C,East York,Woodbine Heights,43.691300,-79.311600,1.0,Convenience Store,Spa,Beer Store,Video Store,Fast Food Restaurant
16,M6C,York,Humewood-Cedarvale,43.691500,-79.430700,1.0,Playground,Field,Trail,Park,Grocery Store
21,M6E,York,Caledonia-Fairbanks,43.688900,-79.450700,1.0,Park,Mexican Restaurant,Beer Store,Sporting Goods Shop,Gym
22,M1G,Scarborough,Woburn,43.765717,-79.221898,1.0,Indian Restaurant,Coffee Shop,Taco Place,Park,Yoga Studio
25,M6G,Downtown Toronto,Christie,43.668300,-79.420500,1.0,Café,Grocery Store,Athletics & Sports,Playground,Candy Store
26,M1H,Scarborough,Cedarbrae,43.768600,-79.238900,1.0,Gaming Cafe,Trail,Lounge,Yoga Studio,Electronics Store
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.665500,-79.437800,1.0,Park,Bakery,Grocery Store,Pizza Place,Bus Line


- rest of clusters are really small and rare

In [63]:
toronto_merged.loc[toronto_merged['Cluster Labels'] > 1]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
27,M2H,North York,Hillcrest Village,43.801500,-79.357700,3.0,Residential Building (Apartment / Condo),Park,Bus Stop,Falafel Restaurant,Eastern European Restaurant
35,M4J,East York,"East Toronto, Broadview North (Old East York)",43.687200,-79.336800,3.0,Park,Convenience Store,Intersection,Farmers Market,Eastern European Restaurant
45,M2L,North York,"York Mills, Silver Hills",43.754700,-79.376400,2.0,Pool,Food Court,Flower Shop,Flea Market,Fish Market
61,M4N,Central Toronto,Lawrence Park,43.730100,-79.393500,3.0,Park,Photography Studio,Yoga Studio,Falafel Restaurant,Dumpling Restaurant
64,M9N,York,Weston,43.706800,-79.517000,3.0,Park,Yoga Studio,Donut Shop,Flower Shop,Flea Market
66,M2P,North York,York Mills West,43.750000,-79.397800,3.0,Convenience Store,Park,Donut Shop,Flower Shop,Flea Market
69,M6P,West Toronto,"High Park, The Junction South",43.660632,-79.464511,3.0,Park,Residential Building (Apartment / Condo),Yoga Studio,Falafel Restaurant,Dumpling Restaurant
71,M1R,Scarborough,"Wexford, Maryvale",43.750700,-79.300300,4.0,Auto Garage,Yoga Studio,Farmers Market,Eastern European Restaurant,Electronics Store
